In [2]:
import pandas as pd
import h2o

ModuleNotFoundError: No module named 'h2o'

In [3]:
#useful functions

def writeList(where, what) :
    with open(where, 'w+') as file :
        for i in what :
            file.write("%s\n" % i)

def readList(where) :
    res = []
    
    with open(where, 'r') as file : 
        for i in file :
            res.append(i[:-1])
    
    return res

zapiszListyPref = '../dataSets/'

In [17]:
merged = pd.read_csv('./dane/merged.csv', index_col = 0)

istotneSkorel = readList('./dane/istotneSkorel.txt')
poKorel05 = readList('./dane/poKorel05')
poKorel07 = readList('./dane/poKorel07')

In [5]:
#puszczamy na poczatek TPOTa

In [7]:
from tpot import TPOTClassifier 

In [29]:
opt1 = TPOTClassifier(generations=20, population_size=50, cv=10,
                                    random_state=42, verbosity=2)

In [30]:
opt1.fit(merged.loc[:, poKorel07], merged.iloc[:, 0])

Generation 1 - Current best internal CV score: 0.7876785714285715
Generation 2 - Current best internal CV score: 0.7876785714285715
Generation 3 - Current best internal CV score: 0.7876785714285715
Generation 4 - Current best internal CV score: 0.8207738095238095
Generation 5 - Current best internal CV score: 0.8207738095238095
Generation 6 - Current best internal CV score: 0.8207738095238095
Generation 7 - Current best internal CV score: 0.8207738095238095
Generation 8 - Current best internal CV score: 0.8207738095238095
Generation 9 - Current best internal CV score: 0.8207738095238095
Generation 10 - Current best internal CV score: 0.8207738095238095
Generation 11 - Current best internal CV score: 0.8207738095238095
Generation 12 - Current best internal CV score: 0.8207738095238095
Generation 13 - Current best internal CV score: 0.8327380952380953
Generation 14 - Current best internal CV score: 0.8327380952380953
Generation 15 - Current best internal CV score: 0.8327380952380953
Gene

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=10,
        disable_update_check=False, early_stop=None, generations=20,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=50,
        random_state=42, scoring=None, subsample=1.0,
        template='RandomTree', use_dask=False, verbosity=2,
        warm_start=False)

In [32]:
opt1.export('dupa.py')

In [43]:
mergedTest = pd.read_csv('mergedTest.csv', index_col=0)
from sklearn.ensemble import GradientBoostingClassifier

# Average CV score on the training set was:0.8327380952380953
optimTree1 = GradientBoostingClassifier(learning_rate=0.1, max_depth=4, max_features=0.1, min_samples_leaf=4, min_samples_split=4, n_estimators=1000, subsample=0.6500000000000001)

In [44]:
optimTree1.fit(merged.loc[:, poKorel07], merged.iloc[:, 0])
results = optimTree1.predict(mergedTest.loc[:, poKorel07])

In [45]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
cm = confusion_matrix(mergedTest.iloc[:, 0], results)

In [47]:
cm

array([[32,  6],
       [11,  1]], dtype=int64)

In [50]:
from sklearn.metrics import roc_auc_score

yProba = optimTree1.predict_proba(mergedTest.loc[:,poKorel07])
yProba = [i[1] for i in yProba]

roc_auc_score(mergedTest.iloc[:,0], yProba)

0.6622807017543859

## h2o based classifier

In [53]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-win64) (build 25.121-b15, mixed mode)
  Starting server from C:\Users\Zuzanna\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Zuzanna\AppData\Local\Temp\tmpot6ojo8h
  JVM stdout: C:\Users\Zuzanna\AppData\Local\Temp\tmpot6ojo8h\h2o_Zuzanna_started_from_python.out
  JVM stderr: C:\Users\Zuzanna\AppData\Local\Temp\tmpot6ojo8h\h2o_Zuzanna_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,Europe/Belgrade
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,4 months and 20 days !!!
H2O cluster name:,H2O_from_python_Zuzanna_fsfbtw
H2O cluster total nodes:,1
H2O cluster free memory:,3.531 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [77]:
trainingFrame = h2o.H2OFrame(merged.loc[:, ['label'] + poKorel07])
validationFrame = h2o.H2OFrame(merged.loc[:, ['label'] + poKorel07])

trainingFrame['label'] = trainingFrame['label'].asfactor()
validationFrame['label'] = validationFrame['label'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [85]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(max_models = 20, seed = 42, max_runtime_secs=3600, nfolds = 5)

In [90]:
aml.train(x = poKorel07, y = 'label', training_frame = trainingFrame)

AutoML progress: |███████████████████████████████████████████████████████ (cancelled) 100%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_b3c59edd7fa0cd7232c2b0e7bc224cd6> was cancelled by the user.

In [91]:
aml.leaderboard

model_id,auc,logloss,mean_per_class_error,rmse,mse
GLM_grid_1_AutoML_20190608_153827_model_1,0.732505,0.470111,0.298174,0.389808,0.15195
GLM_grid_1_AutoML_20190608_154603_model_1,0.732505,0.470111,0.298174,0.389808,0.15195
DeepLearning_grid_1_AutoML_20190608_154603_model_1,0.711714,0.691822,0.2893,0.443743,0.196908
GBM_1_AutoML_20190608_154603,0.71146,0.601031,0.302231,0.422416,0.178435
GBM_1_AutoML_20190608_153827,0.71146,0.601031,0.302231,0.422416,0.178435
DeepLearning_grid_1_AutoML_20190608_154603_model_5,0.70715,1.27471,0.327586,0.458794,0.210492
GBM_grid_1_AutoML_20190608_154603_model_1,0.7036,0.525983,0.317698,0.415406,0.172562
GBM_grid_1_AutoML_20190608_153827_model_1,0.7036,0.525983,0.317698,0.415406,0.172562
DeepLearning_grid_1_AutoML_20190608_154603_model_6,0.700177,1.93272,0.321755,0.48617,0.236362
DeepLearning_grid_1_AutoML_20190608_153827_model_3,0.698783,1.12533,0.310852,0.47432,0.22498


In [92]:
topmodel = aml.leader

In [93]:
topmodel.model_performance(test_data = validationFrame, valid = True)


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.13136048486775564
RMSE: 0.36243687018259557
LogLoss: 0.41466350003220437
Null degrees of freedom: 149
Residual degrees of freedom: 117
Null deviance: 160.56514821830135
Residual deviance: 124.39905000966134
AIC: 190.39905000966132
AUC: 0.8400101419878296
pr_auc: 0.5989284972067035
Gini: 0.6800202839756593
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.27022807833653034: 


,0,1,Error,Rate
0,97.0,19.0,0.1638,(19.0/116.0)
1,9.0,25.0,0.2647,(9.0/34.0)
Total,106.0,44.0,0.1867,(28.0/150.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2702281,0.6410256,43.0
max f2,0.2246831,0.7291667,55.0
max f0point5,0.3642644,0.6147541,21.0
max accuracy,0.4086470,0.8266667,13.0
max precision,0.8169928,1.0,0.0
max recall,0.1217628,1.0,111.0
max specificity,0.8169928,1.0,0.0
max absolute_mcc,0.2702281,0.5255405,43.0
max min_per_class_accuracy,0.2431406,0.7931034,50.0
max mean_per_class_accuracy,0.2431406,0.7936105,50.0


Gains/Lift Table: Avg response rate: 22,67 %, avg score: 22,67 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0133333,0.7987516,4.4117647,4.4117647,1.0,0.8111591,1.0,0.8111591,0.0588235,0.0588235,341.1764706,341.1764706
,2,0.02,0.6248418,4.4117647,4.4117647,1.0,0.7919095,1.0,0.8047426,0.0294118,0.0882353,341.1764706,341.1764706
,3,0.0333333,0.5779935,2.2058824,3.5294118,0.5,0.6002740,0.8,0.7229552,0.0294118,0.1176471,120.5882353,252.9411765
,4,0.04,0.5521584,4.4117647,3.6764706,1.0,0.5767280,0.8333333,0.6985840,0.0294118,0.1470588,341.1764706,267.6470588
,5,0.0533333,0.5251175,2.2058824,3.3088235,0.5,0.5465233,0.75,0.6605688,0.0294118,0.1764706,120.5882353,230.8823529
,6,0.1,0.4046911,3.1512605,3.2352941,0.7142857,0.4572202,0.7333333,0.5656728,0.1470588,0.3235294,215.1260504,223.5294118
,7,0.1533333,0.3539762,2.2058824,2.8772379,0.5,0.3820264,0.6521739,0.5017958,0.1176471,0.4411765,120.5882353,187.7237852
,8,0.2,0.3157652,1.8907563,2.6470588,0.4285714,0.3345485,0.6,0.4627714,0.0882353,0.5294118,89.0756303,164.7058824
,9,0.3,0.2605502,2.0588235,2.4509804,0.4666667,0.2873075,0.5555556,0.4042834,0.2058824,0.7352941,105.8823529,145.0980392
,10,0.4,0.2145606,0.8823529,2.0588235,0.2,0.2358467,0.4666667,0.3621742,0.0882353,0.8235294,-11.7647059,105.8823529


In [82]:
topmodel.plot

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_grid_1_AutoML_20190608_153827_model_1


ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.13136048486775564
RMSE: 0.36243687018259557
LogLoss: 0.41466350003220437
Null degrees of freedom: 149
Residual degrees of freedom: 117
Null deviance: 160.56514821830135
Residual deviance: 124.39905000966134
AIC: 190.39905000966132
AUC: 0.8400101419878296
pr_auc: 0.5989284972067035
Gini: 0.6800202839756593
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.27022807833653034: 


,0,1,Error,Rate
0,97.0,19.0,0.1638,(19.0/116.0)
1,9.0,25.0,0.2647,(9.0/34.0)
Total,106.0,44.0,0.1867,(28.0/150.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2702281,0.6410256,43.0
max f2,0.2246831,0.7291667,55.0
max f0point5,0.3642644,0.6147541,21.0
max accuracy,0.4086470,0.8266667,13.0
max precision,0.8169928,1.0,0.0
max recall,0.1217628,1.0,111.0
max specificity,0.8169928,1.0,0.0
max absolute_mcc,0.2702281,0.5255405,43.0
max min_per_class_accuracy,0.2431406,0.7931034,50.0
max mean_per_class_accuracy,0.2431406,0.7936105,50.0


Gains/Lift Table: Avg response rate: 22,67 %, avg score: 22,67 %



...


Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
accuracy,0.7333333,0.0596285,0.6333333,0.8333333,0.8333333,0.6666667,0.7
auc,0.7561485,0.0569576,0.6335404,0.7986111,0.8541667,0.6931818,0.8012422
err,0.2666667,0.0596285,0.3666667,0.1666667,0.1666667,0.3333333,0.3
err_count,8.0,1.7888544,11.0,5.0,5.0,10.0,9.0
f0point5,0.5098019,0.0541584,0.3968254,0.5952381,0.5952381,0.46875,0.4929577
---,---,---,---,---,---,---,---
r2,0.1551481,0.0596506,0.0768301,0.1630346,0.2970057,0.0615058,0.1773640
recall,0.8261905,0.0697420,0.7142857,0.8333333,0.8333333,0.75,1.0
residual_deviance,27.561232,2.6418767,30.58864,25.491737,22.247938,32.791084,26.686764
rmse,0.3839438,0.0227212,0.4063802,0.3659432,0.335379,0.4284014,0.3836150



See the whole table with table.as_data_frame()
Scoring History: 


,timestamp,duration,iteration,lambda,predictors,deviance_train,deviance_test,deviance_xval,deviance_se
,2019-06-08 15:38:29,0.000 sec,2,",11E2",33,1.0344318,nan,1.0477509,0.0323961
,2019-06-08 15:38:29,0.004 sec,4,",83E1",33,1.0233872,nan,1.0400595,0.0329251
,2019-06-08 15:38:29,0.004 sec,6,",61E1",33,1.0098131,nan,1.0305932,0.0336800
,2019-06-08 15:38:29,0.010 sec,8,",44E1",33,0.9937094,nan,1.0193182,0.0347317
,2019-06-08 15:38:29,0.010 sec,10,",32E1",33,0.9753261,nan,1.0065104,0.0361641
,2019-06-08 15:38:29,0.014 sec,12,",23E1",33,0.9551774,nan,0.9927091,0.0380803
,2019-06-08 15:38:29,0.014 sec,14,",17E1",33,0.9339062,nan,0.9787811,0.0405930
,2019-06-08 15:38:30,0.018 sec,16,",12E1",33,0.9122330,nan,0.9656753,0.0437850
,2019-06-08 15:38:30,0.018 sec,18,",9E0",33,0.8907034,nan,0.9544555,0.0477310
,2019-06-08 15:38:30,0.024 sec,20,",66E0",33,0.8696609,nan,0.9460320,0.0524524


<bound method H2OBinomialModel.plot of >

In [84]:
h2o.save_model(topmodel, 'topModl2')

'C:\\Users\\Zuzanna\\Desktop\\biohack\\predicting_response\\topModl\\GLM_grid_1_AutoML_20190608_153827_model_1'